<a href="https://colab.research.google.com/github/socd06/databricks-language-hackathon/blob/main/June6_Dolly_3B_Student_Advisor_with_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate einops Xformers langchain InstructorEmbedding sentence-transformers chromadb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.7/993.7 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 k

In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install unstructured pandoc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 79.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 kB 

In [ ]:
# CLEAR CUDA
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [3]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer

model_name = "databricks/dolly-v2-3b"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")

generate_text = pipeline(model=model_name, torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [4]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
import torch

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

# template for an instrution with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt)

In [5]:
hf_pipeline

HuggingFacePipeline(cache=None, verbose=False, callbacks=None, callback_manager=None, pipeline=<transformers_modules.databricks.dolly-v2-3b.e19a5252f69d79d94ac95045eb9b8a158775f701.instruct_pipeline.InstructionTextGenerationPipeline object at 0x7f5baf73e8c0>, model_id='gpt2', model_kwargs=None, pipeline_kwargs=None)

In [ ]:
# TODO: Check if input is checked somewhere, otherwise check something like this:

# Your output sequence
output_sequence = "This is an example output sequence."

# Tokenize the output sequence
tokenized_output = tokenizer.encode(output_sequence, return_tensors="pt")

# Count the number of tokens in the tokenized sequence
num_tokens = tokenized_output.shape[1]

print(f"Number of tokens in the output sequence: {num_tokens}")

Number of tokens in the output sequence: 7


# Test LLM Chain

In [6]:
question = 'Who was Dolly the sheep?'
llm_chain.run(question)

'\nDolly the sheep was a sheep born in 1929. She was the first cloned mammal and the first successful hermaphrodite birth in the world.'

In [7]:
question = 'what are the colors of the rainbow?'
llm_chain.run(question)

'\nred, orange, yellow, green, blue, indigo, and purple'

# Prepare Documents


In [9]:
!pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

In [11]:
from langchain.document_loaders.sitemap import SitemapLoader


In [12]:
import os
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

In [13]:
sitemap_loader = SitemapLoader(web_path="https://www.uwo.ca/sitemap.xml")

docs = sitemap_loader.load()

Fetching pages: 100%|##########| 63/63 [00:04<00:00, 13.72it/s]


In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs)

In [ ]:
# DOWNLOAD Workable template employee handbook from: https://resources.workable.com/wp-content/uploads/2017/09/Employee-Handbook.pdf

# langchain TUTORIAL WAY
# Load and process the PDF files
loader = DirectoryLoader('/content/docs/', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-14-44c67e7b3ff4>:2 in <cell line: 2>                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'documents' is not defined

In [16]:
len(texts)

598

In [17]:
texts[2]

Document(page_content="Popular Pages\n\nContinuing Studies\nHousing & Residence\nOffice of the Registrar\n\n\n  \n\nExplore Western's Campus\n\n\n\n\n \nAcademics \n\n\nAcademics\n\nDepartments. Faculties. Affiliates.\nUndergraduate Programs\nGraduate Programs\nContinuing Studies\nDistance Studies\nInternational Learning\n\n\n\nFaculties / Schools\n\nArts & Humanities\nDon Wright Faculty of Music\nEducation\nEngineering\nGraduate & Postdoctoral Studies\nHealth Sciences\n\n\n \n\nInformation & Media Studies\nLaw\nIvey Business School\nSchulich Medicine & Dentistry\nScience\nSocial Science\n\n\n\nPopular Pages\n\nAcademic Calendar\nCourse Timetable\nWestern Libraries\nOffice of the Registrar\nConvocation\n\n\n\n\n \nCampus Life\n\n\nCampus A-Z\n\nAcademics & Learning\nArts & Culture\nAthletics & Recreation\nCareer Services\nClubs & Associations\n\n\n \n\nComputer Support\nDiversity\nFamily Support\nFinancial Services\nFood & Dining\n\n\n \n\nHealth & Wellness\nCampus Safety\nShopping & R

# Download HuggingFace Embeddings
Check [MTEB English Leaderboard](https://huggingface.co/spaces/mteb/leaderboard) to make sure you download embeddings with good performance

In [18]:
# Choose one of the top performers from the MTEB English Leaderboard

from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

# top #2 when task = Retrieval June 2023 for under ~500 MB
model_name = "intfloat/e5-base-v2" 

hf = HuggingFaceEmbeddings(model_name=model_name)

# Make a Vector Database

In [19]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = hf 

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

# and a retriever

In [20]:
retriever = vectordb.as_retriever(search_kwargs={'k':2})

# Test the basic retriever

In [23]:
docs = retriever.get_relevant_documents("How much is tuition?")

In [24]:
docs[0]

Document(page_content='Faculties / Schools\n\nArts & Humanities\nDon Wright Faculty of Music\nEducation\nEngineering\nGraduate & Postdoctoral Studies\nHealth Sciences\n\n\n \n\nInformation & Media Studies\nLaw\nIvey Business School\nSchulich Medicine & Dentistry\nScience\nSocial Science\n\n\n\nPopular Pages\n\nAcademic Calendar\nCourse Timetable\nWestern Libraries\nOffice of the Registrar\nConvocation\n\n\n\n\n \nCampus Life\n\n\nCampus A-Z\n\nAcademics & Learning\nArts & Culture\nAthletics & Recreation\nCareer Services\nClubs & Associations\n\n\n \n\nComputer Support\nDiversity\nFamily Support\nFinancial Services\nFood & Dining\n\n\n \n\nHealth & Wellness\nCampus Safety\nShopping & Retail\nStudent Experience\nTransportation & Parking\n\n\n\nPopular Pages\n\nThe Bookstore\nCampus Recreation\nFacilities Management\nHousing & Residence\nInformation Technology\n\n\n\n\n\nResearch\n\n\nMENU\n\nAbout Western\nAdmission\nAcademics\nCampus Life\nResearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n Re

# Make a proper Question Retrieval chain

In [25]:
qa_chain = RetrievalQA.from_chain_type(llm=hf_pipeline, 
                                       chain_type="stuff", 
                                       retriever=retriever, 
                                       return_source_documents=True)

In [26]:
qa_chain

RetrievalQA(memory=None, callbacks=None, callback_manager=None, verbose=False, combine_documents_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:", template_format='f-string', validate_template=True), llm=HuggingFacePipeline(cache=None, verbose=False, callbacks=None, callback_manager=None, pipeline=<transformers_modules.databricks.dolly-v2-3b.e19a5252f69d79d94ac95045eb9b8a158775f701.instruct_pipeline.InstructionTextGenerationPipeline object at 0x7f5baf73e8c0>, model_id='gpt2', m

In [39]:
# try to set the tone to "as a human-like Student Advisor"
qa_chain.combine_documents_chain.llm_chain.prompt.template = '''
You are a prospective student advisor for a university. 
Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always answer with unbiased, ethical and safe advise.
----------------
{context}

Question: {question}
Helpful Answer:'''

In [28]:
qa_chain.combine_documents_chain.llm_chain.prompt.template

"\nYou are a prospective student advisor.\nUse the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nAlways answer with unbiased, ethical and safe advise.\n----------------\n{context}\n\nQuestion: {question}\nHelpful Answer:"

In [31]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
      print(source.metadata['source'])

# Test QA Retrieving Chain

In [38]:
query = "How much is tuition for a bachelor's program as opposed to a graduate program?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


Tuition for undergraduate programs at Western is $50,600. Tuition for graduate programs is $109,800.


Sources:
https://www.uwo.ca/index.html
https://www.uwo.ca/western-remembers-flight752-farsi.html


In [33]:
query = "Recommend me an engineering program"
llm_response = qa_chain(query)
process_llm_response(llm_response)


There are many options when looking at Engineering programs at Western University. As the institution is Canada's first University, they offer a wide range of subjects and specializations. From mechanical, electrical, chemical and biochemical, to biological, agricultural and environmental engineering.


Sources:
https://www.uwo.ca/index.html
https://www.uwo.ca/community/summer-camps.html


In [34]:
query = "I want to apply to law school, when is the deadline?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


It depends how you apply. In general, law school applications are reviewed in fall for fall semesters, in spring for the spring semesters, and in summer for summer school. There may be a rolling admissions program for specific schools, and/or some schools allow an online application, which may have less of an application deadline. Contact the law school for more information.


Sources:
https://www.uwo.ca/faculty_staff/awards/nomination_selection.html
https://www.uwo.ca/future_students/index.html


# Make a Better Question Retrieval chain using Memory

There is a bug when using both return_source_documents and ConversationBufferMemory

[ConversationalRetrievalChain unable to use ConversationBufferMemory when return_source_documents=True](https://github.com/hwchase17/langchain/issues/5630)

So we define a custom class as a workaround

In [57]:
from langchain.memory import ConversationBufferMemory

from typing import Dict, Any

class AnswerConversationBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        return super(AnswerConversationBufferMemory, self).save_context(inputs,{'response': outputs['result']})

In [58]:
qa_chain_with_memory = RetrievalQA.from_chain_type(llm=hf_pipeline, 
                                                   chain_type="stuff", 
                                                   retriever=retriever, 
                                                   return_source_documents=True,
                                                   memory=AnswerConversationBufferMemory())

In [59]:
# try to set the tone to "as a human-like Student Advisor"
qa_chain_with_memory.combine_documents_chain.llm_chain.prompt.template = '''
You are a prospective student advisor for a university. 
Use the following pieces of context to answer the users question. 
If you don't know the answer, ask for more context or say that you don't know, don't try to make up an answer. 
Always answer with unbiased, ethical and safe advise.
----------------
{context}

Question: {question}
Helpful Answer:'''

In [60]:
query = "How much is tuition in engineering?"
llm_response = qa_chain_with_memory(query)
process_llm_response(llm_response)


Western offers tuition-free higher education in the engineering disciplines at several Canadian universities. Tuition fees apply to the other disciplines at Western. Learn more.


Sources:
https://www.uwo.ca/western-remembers-flight752-farsi.html
https://www.uwo.ca/western-remembers-flight752-farsi.html


In [61]:
query = "How much is tuition in med school?"
llm_response = qa_chain_with_memory(query)
process_llm_response(llm_response)


Med School Tuition varies widely based on where you go to school. Western has been surveyed at over $280,000 USD.


Sources:
https://www.uwo.ca/western-remembers-flight752-farsi.html
https://www.uwo.ca/western-remembers-flight752-farsi.html


In [67]:
query = "I am interested in both medicine and engineering, do you have a program that is a mix of the two?"
llm_response = qa_chain_with_memory(query)
process_llm_response(llm_response)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Yes, the Department of Mechanical and Materials Engineering has a Biomedical Engineering program within it.


Sources:
https://www.uwo.ca/a-z.html
https://www.uwo.ca/severe-weather-closure-faqs.html


# Trying out Knowledge Graph Memory

In [71]:
# workaround again
class AnswerKnowledgeGraphMemory(ConversationKGMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        return super(AnswerKnowledgeGraphMemory, self).save_context(inputs,{'response': outputs['result']})

In [98]:
from langchain.memory import ConversationKGMemory

memory = AnswerKnowledgeGraphMemory(llm=hf_pipeline)

qa_chain_with_kg_memory = RetrievalQA.from_chain_type(llm=hf_pipeline,
                                                      chain_type="stuff", 
                                                      retriever=retriever, 
                                                      return_source_documents=True,                                                      
                                                      memory=memory)

In [99]:
# try to set the tone to "as a human-like Student Advisor"
template = '''
You are a prospective student advisor for a university. You are talkative and provide lots of specific details using the context given by the users question. 
If you don't know the answer, you truthfully say you don't know and don't try to make up an answer. You use the information contained in the "Relevant Information" section and You always answer with unbiased, ETHICAL and safe advise.

Relevant Information:
{history}

Conversation:

Question: {question}
Helpful Answer:'''

prompt = PromptTemplate(
    input_variables=["history", "question"], template=template)

In [88]:
qa_chain_with_kg_memory.combine_documents_chain.llm_chain.prompt = prompt
qa_chain_with_kg_memory.combine_documents_chain.llm_chain.prompt

PromptTemplate(input_variables=['history', 'question'], output_parser=None, partial_variables={}, template='\nYou are a prospective student advisor for a university. You are talkative and provide lots of specific details using the following pieces of context to answer the users question. \nIf you don\'t know the answer, you truthfully say you don\'t know and don\'t try to make up an answer. You use the information contained in the "Relevant Information" section and do NOT hallucinate.\nYou always answer with unbiased, ETHICAL and safe advise.\n\nRelevant Information:\n{history}\n\nConversation:\n\nQuestion: {question}\nHelpful Answer:', template_format='f-string', validate_template=True)

In [92]:
qa_chain_with_kg_memory.combine_documents_chain

StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=PromptTemplate(input_variables=['history', 'question'], output_parser=None, partial_variables={}, template='\nYou are a prospective student advisor for a university. You are talkative and provide lots of specific details using the following pieces of context to answer the users question. \nIf you don\'t know the answer, you truthfully say you don\'t know and don\'t try to make up an answer. You use the information contained in the "Relevant Information" section and do NOT hallucinate.\nYou always answer with unbiased, ETHICAL and safe advise.\n\nRelevant Information:\n{history}\n\nConversation:\n\nQuestion: {question}\nHelpful Answer:', template_format='f-string', validate_template=True), llm=HuggingFacePipeline(cache=None, verbose=False, callbacks=None, 

In [100]:
query = "I am interested in both medicine and engineering, do you have a program that is a mix of the two?"
llm_response = qa_chain_with_kg_memory(query)
process_llm_response(llm_response)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [103]:
query = "I like biomedical engineering, do you offer it as a graduate program?"
llm_response = qa_chain_with_kg_memory(query)
process_llm_response(llm_response)


Yes, Biomedical Engineering is offered as a graduate program at Western. 
However, there are many factors that need to be considered before making a decision to pursue a graduate degree. 
Questions you should ask:
- Does the graduate program offer cooperative education or a specific 
program to build workforce skills?
- Is the graduate program offered online? 
- Does the graduate program offer tuition reimbursement?
- Are there industry partnerships? 
- Does the graduate program allow you to work on a clinical trial?


Sources:
https://www.uwo.ca/a-z.html
https://www.uwo.ca/faculty_staff/awards/twae_archive.html


In [104]:
query = "How much is tuition for this program?"
llm_response = qa_chain_with_kg_memory(query)
process_llm_response(llm_response)


Each year, we admit students into our Undergraduate Program for approximately $40,000. Although there are discounts and scholarships available, the price of tuition to Western is approximately $58,000 per year.


Sources:
https://www.uwo.ca/index.html
https://www.uwo.ca/community/summer-camps.html


In [105]:
query = "WOw, expensive! is this the fee for international students?"
llm_response = qa_chain_with_kg_memory(query)
process_llm_response(llm_response)


Western's tuition and fees are among the lowest of the 25 California public universities.
You are more than covered with state aid, including:
$37,000 for resident students
$45,000 for non-residents
$13,600 for dependent children


Sources:
https://www.uwo.ca/future_students/index.html
https://www.uwo.ca/index.html


# Notes for next version
- Make a National Building Code chatbot 
- Make a school board chatbot
- See if 7B model can be handled by free DataBricks
- Figure out how to handle data in databricks using dbfs ([Dolly Example](https://www.dbdemos.ai/demo-notebooks.html?demoName=llm-dolly-chatbot))